In [1]:
#!pip install selenium

In [2]:
#!pip install bs4

In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


from bs4 import BeautifulSoup

import requests
import pandas as pd

#### 1. Access search box

In [15]:
site = 'https://wuzzuf.net/jobs/egypt'
# driver = webdriver.Chrome()
driver=webdriver.Firefox()
driver.get(site)
driver.maximize_window()
WebDriverWait(driver,timeout=4)


<selenium.webdriver.support.wait.WebDriverWait (session="9e78f55b-ed56-43f4-a8d5-240d3e90eb80")>

In [16]:

search_box = driver.find_element(By.CLASS_NAME,'search-bar-input')

search_box.clear()
search_box.send_keys('AI Engineer'+ Keys.ENTER)


In [18]:
driver.switch_to.window(driver.window_handles[-1])
first_page=driver.current_url

first_page

'https://wuzzuf.net/search/jobs/?q=AI+Engineer&a=hpb'

In [ ]:
driver.page_source

In [ ]:
soup = BeautifulSoup(driver.page_source, 'lxml')

In [ ]:
soup

In [ ]:
soup.find('a[href^="https://wuzzuf.net/jobs/p/"')


In [ ]:
home = 'https://wuzzuf.net'


def filter_tags(tag):
    return tag.has_attr('href') and  tag.get('href').startswith('/jobs/p/')
    
links= []
for i in soup.find_all(filter_tags):
    links.append(home + i.get('href'))
            

In [ ]:
links

In [ ]:
# driver.get(links[0])
# s1=BeautifulSoup(driver.page_source, 'lxml')
def get_data(page_content):
    """
    page_content: soup
    """

    
    #get title data
    #1.get title section tag that contain basic info in html page
    title_section = page_content.find_all('section',class_ = 'css-dy1y6u')
    
    #2. get the divs that contain the required data
    divs = title_section[0].find('div',class_='css-bjn8wh' )

    #3. get job title 
    title=divs.find('h1')
    job_title=title.text
    
    #4. get job type
    job_type=title_section[0].find('span')
    Job_type=job_type.text
    
    #5. get company name 
    company=title_section[0].find('a',class_="css-p7pghv")
    company_name=company.text
    
    #6. get company location
    location=title_section[0].find('strong',class_='css-9geu3q')
    location.div.decompose()
    Company_location=location.text
    #create dictionary for the previous data 
    d1={ 'job title':[job_title],
       'job_type':[Job_type],
       'company':[company_name],
       'location':[Company_location]

    }
    df1=pd.DataFrame(d1)

    

    ###############################################################
    ##get data of job details section
    #1. get section that contain job details in html page 
    
    sections = page_content.find_all('section',class_ = 'css-3kx5e2')

    divs = sections[0].find_all('div')

    job_details = []
    tr = ['Experience Needed:','Career Level:', 'Education Level:', 'Salary:', 'Job Categories:', 'skills and tools' ]
    for div in divs:
        spans = div.find_all('span')

        temp =  set()
        for s in spans:
            if s.text in tr:
                continue
            temp.add(s.text)

        job_details.append([str(temp).replace('{','').replace('}','')])


    df2= pd.DataFrame(columns=tr)

    
    d2 = dict(zip(tr,job_details))
    
    test_df = pd.DataFrame(data = d2 )
    df2=pd.concat([df2,test_df],axis=0)


    final_df=pd.concat([df1,df2],axis=1)
    return final_df



In [ ]:
# get_data(s1).columns


In [ ]:
jobs_df=pd.DataFrame(columns=['job title', 'job_type', 'company', 'location', 'Experience Needed:',
       'Career Level:', 'Education Level:', 'Salary:', 'Job Categories:',
       'skills and tools'])
jobs_df

In [ ]:
for link in links:
    driver.get(link)
    soup1=BeautifulSoup(driver.page_source, 'lxml')
    df=get_data(soup1)
    jobs_df=pd.concat([jobs_df,df],axis=0)


In [ ]:
jobs_df

In [ ]:
jobs_df.reset_index(inplace=True ,drop=True)

In [ ]:
jobs_df

In [7]:
first_page

'https://wuzzuf.net/jobs/egypt'

In [66]:
###scroll between pages
driver.get(first_page)


buttons=driver.find_element(By.CLASS_NAME,'css-if9uys')
Buttons=buttons.find_elements(By.TAG_NAME,'button')


In [67]:
Buttons[0]

<selenium.webdriver.remote.webelement.WebElement (session="9e78f55b-ed56-43f4-a8d5-240d3e90eb80", element="fa8eb8b8-ec30-4af2-a7f5-b9575cb34efc")>

In [68]:
for i in range(len( Buttons)-1):
    Buttons[i].click()
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(driver.current_url)
    next_button=driver.find_element(By.XPATH,Xpath)




StaleElementReferenceException: Message: The element reference of <button class="css-1dtj1k4 ezfki8j0" type="button"> is stale; either the element is no longer attached to the DOM, it is not in the current frame context, or the document has been refreshed
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:183:5
StaleElementReferenceError@chrome://remote/content/shared/webdriver/Errors.jsm:464:5
element.resolveElement@chrome://remote/content/marionette/element.js:681:11
evaluate.fromJSON@chrome://remote/content/marionette/evaluate.js:254:26
evaluate.fromJSON@chrome://remote/content/marionette/evaluate.js:262:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.jsm:79:29


In [14]:
driver.close()